Experimento:
Buscamos lograr contestar la pregunta:

¿Es posible asociar el valor de alguna característica de la música como su energía o su tempo con un lugar geográfico?


In [31]:
import pandas as pd
import numpy as np


#juntar paises en base al continente
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [17]:
df = pd.read_excel('C:/Users/diego/OneDrive/Escritorio/universidad/semestre10/mineria/T2/Insight_Alchemist/Spotify.xlsx')

#Ajustamos columnas para contraarrestar error de formato en archivo xlsx
df['duration_ms'] = df['duration_ms']/10
df['popularity'] = df['popularity']/10
df['streams'] = df['streams']/10
df['af_danceability'] = df['af_danceability']/1000
df['af_energy'] = df['af_energy']/1000
df['af_key'] = df['af_key']/10
df['af_loudness'] = df['af_loudness']/1000
df['af_speechiness'] = df['af_speechiness']/1000
df['af_acousticness'] = df['af_acousticness']/1000
df['af_instrumentalness'] = df['af_instrumentalness']/1000
df['af_liveness'] = df['af_liveness']/1000
df['af_valence'] = df['af_valence']/1000
df['af_tempo'] = df['af_tempo']/1000
df['af_time_signature'] = df['af_time_signature']/10

Primero modificamos la información de la columna regiones para reducir las clases al agrupar los países por regiones geográficas más amplia.

In [ ]:
d1 = [
    (['Morocco','South Africa'],'Africa'),
    (['Argentina','Bolivia','Brazil','Chile','Colombia','Costa Rica','Dominican Republic','Ecuador','El Salvador','Guatemala','Honduras','Mexico','Nicaragua','Panama','Paraguay','Peru','Uruguay'],'America_latina'),
    (['Canada','United States'],'America_norte'),
    (['Hong Kong','India','Indonesia','Japan','Malaysia','Philippines','Singapore','Taiwan','Thailand','Vietnam'],'Asia'),
    (['Andorra','Austria','Belgium','Bulgaria','Czech Republic','Denmark','Estonia','Finland','France','Germany','Greece','Hungary','Iceland','Ireland','Italy','Latvia','Lithuania','Luxembourg',
     'Netherlands','Norway','Poland','Portugal','Romania','Russia','Slovakia','Spain','Sweden','Switzerland','Ukraine','United Kingdom'],'Europa'),
    (['Australia','New Zealand'],'Oceania'),
    (['Egypt','Israel','Saudi Arabia','Turkey','United Arab Emirates'],'Oriente_medio')
]

def apply_etiqueta(elemento):
    for i in d1:
        if elemento in i[0]:
            return i[1]
        
new_reg = df["region"]


for index, row in df.iterrows():
    new_data = apply_etiqueta(row["region"])
    new_reg.iat[index] = new_data

#se reemplaza la columa de region
df["region"] = new_reg


Para encontrar si esta relación existe vamos a usar un Decision Tree Classifier.

Primero vamos atributo por atributo probando para ver cual tiene mejores resultados.

In [39]:
for i in ['af_danceability','af_energy', 'af_key', 'af_loudness', 'af_mode', 'af_speechiness','af_acousticness', 'af_liveness', 'af_valence','af_tempo', 'af_time_signature']:

    df = df.dropna(subset=["region", i])  # Drop rows where target or feature is NaN
    X = df[i].copy().values.reshape(-1, 1)
    y = df["region"]

    #primero separamos los datos de entrenamiento y validacion/test
    X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(X,y,test_size=0.3,random_state=0,stratify=y)

    # Luego separamos los datos de validación y pruebas                                       0.5 x 0.3 = 0.15
    X_val, X_test, y_val, y_test = train_test_split(X_val_and_test, y_val_and_test, test_size=0.5, random_state=0, stratify=y_val_and_test)

    #estandarizacion de datos, transformar los datos de manera que tengan media 0 y desviacion estandar 1.
    std_scaler = StandardScaler()
    X_train_std_scaled = std_scaler.fit_transform(X_train)

    X_val_std_scaled = std_scaler.transform(X_val)
    X_test_std_scaled = std_scaler.transform(X_test)

    #definimos nuestro modelo
    clf = DecisionTreeClassifier(criterion="gini",random_state=0)

    #entrenamos al modelo con fit
    clf.fit(X_train_std_scaled, y_train)   ## Entrenar usando X (features), y (clase)

    #realizamos predicciones de nuestros datos con los datos de validación
    y_val_pred = clf.predict(X_val_std_scaled)


    #metricas de las predicciones
    print(i)
    print(classification_report(y_val, y_val_pred))

af_danceability
                precision    recall  f1-score   support

        Africa       0.42      0.02      0.03      1785
America_latina       0.53      0.50      0.52     15151
 America_norte       0.88      0.00      0.01      1785
          Asia       0.52      0.26      0.35      8921
        Europa       0.51      0.80      0.62     25448
       Oceania       0.53      0.01      0.03      1785
 Oriente_medio       0.46      0.04      0.07      4462

      accuracy                           0.51     59337
     macro avg       0.55      0.23      0.23     59337
  weighted avg       0.52      0.51      0.46     59337

af_energy
                precision    recall  f1-score   support

        Africa       0.56      0.02      0.03      1785
America_latina       0.55      0.53      0.54     15151
 America_norte       0.31      0.01      0.02      1785
          Asia       0.51      0.28      0.36      8921
        Europa       0.52      0.80      0.63     25448
       Oceania    

c:\Users\diego\OneDrive\Escritorio\universidad\semestre10\mineria\T2\Insight_Alchemist\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\diego\OneDrive\Escritorio\universidad\semestre10\mineria\T2\Insight_Alchemist\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\diego\OneDrive\Escritorio\universidad\semestre10\mineria\T2\Insight_Alchemist\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in la

                precision    recall  f1-score   support

        Africa       0.00      0.00      0.00      1785
America_latina       0.00      0.00      0.00     15151
 America_norte       0.00      0.00      0.00      1785
          Asia       0.00      0.00      0.00      8921
        Europa       0.43      1.00      0.60     25448
       Oceania       0.00      0.00      0.00      1785
 Oriente_medio       0.00      0.00      0.00      4462

      accuracy                           0.43     59337
     macro avg       0.06      0.14      0.09     59337
  weighted avg       0.18      0.43      0.26     59337

af_loudness
                precision    recall  f1-score   support

        Africa       0.79      0.39      0.52      1785
America_latina       0.75      0.71      0.73     15151
 America_norte       0.55      0.12      0.20      1785
          Asia       0.75      0.57      0.65      8921
        Europa       0.64      0.88      0.74     25448
       Oceania       0.67      0

c:\Users\diego\OneDrive\Escritorio\universidad\semestre10\mineria\T2\Insight_Alchemist\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\diego\OneDrive\Escritorio\universidad\semestre10\mineria\T2\Insight_Alchemist\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\diego\OneDrive\Escritorio\universidad\semestre10\mineria\T2\Insight_Alchemist\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in la

                precision    recall  f1-score   support

        Africa       0.00      0.00      0.00      1785
America_latina       0.00      0.00      0.00     15151
 America_norte       0.00      0.00      0.00      1785
          Asia       0.00      0.00      0.00      8921
        Europa       0.43      1.00      0.60     25448
       Oceania       0.00      0.00      0.00      1785
 Oriente_medio       0.00      0.00      0.00      4462

      accuracy                           0.43     59337
     macro avg       0.06      0.14      0.09     59337
  weighted avg       0.18      0.43      0.26     59337

af_speechiness
                precision    recall  f1-score   support

        Africa       0.50      0.07      0.13      1785
America_latina       0.56      0.51      0.54     15151
 America_norte       0.24      0.00      0.00      1785
          Asia       0.49      0.25      0.33      8921
        Europa       0.52      0.82      0.64     25448
       Oceania       0.63    

c:\Users\diego\OneDrive\Escritorio\universidad\semestre10\mineria\T2\Insight_Alchemist\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\diego\OneDrive\Escritorio\universidad\semestre10\mineria\T2\Insight_Alchemist\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\diego\OneDrive\Escritorio\universidad\semestre10\mineria\T2\Insight_Alchemist\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in la

                precision    recall  f1-score   support

        Africa       0.00      0.00      0.00      1785
America_latina       0.00      0.00      0.00     15151
 America_norte       0.00      0.00      0.00      1785
          Asia       0.00      0.00      0.00      8921
        Europa       0.43      1.00      0.60     25448
       Oceania       0.00      0.00      0.00      1785
 Oriente_medio       0.00      0.00      0.00      4462

      accuracy                           0.43     59337
     macro avg       0.06      0.14      0.09     59337
  weighted avg       0.18      0.43      0.26     59337



De lo anterior se pude observar que las dos características que presentan mayor relación son loudness y tempo.

Ahora probamos con todas las características a la vez para ver si se encuentra un mejor resultado.

In [ ]:
df = df.dropna(subset=["region", i])  # Drop rows where target or feature is NaN
X = df[['af_danceability','af_energy', 'af_key', 'af_loudness', 'af_mode', 'af_speechiness','af_acousticness', 'af_liveness', 'af_valence','af_tempo', 'af_time_signature']].copy()
y = df["region"]

#primero separamos los datos de entrenamiento y validacion/test
X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(X,y,test_size=0.3,random_state=0,stratify=y)

# Luego separamos los datos de validación y pruebas                                       0.5 x 0.3 = 0.15
X_val, X_test, y_val, y_test = train_test_split(X_val_and_test, y_val_and_test, test_size=0.5, random_state=0, stratify=y_val_and_test)

#estandarizacion de datos, transformar los datos de manera que tengan media 0 y desviacion estandar 1.
std_scaler = StandardScaler()
X_train_std_scaled = std_scaler.fit_transform(X_train)

X_val_std_scaled = std_scaler.transform(X_val)
X_test_std_scaled = std_scaler.transform(X_test)

#definimos nuestro modelo
clf = DecisionTreeClassifier(criterion="gini",random_state=0)

#entrenamos al modelo con fit
clf.fit(X_train_std_scaled, y_train)   ## Entrenar usando X (features), y (clase)

#realizamos predicciones de nuestros datos con los datos de validación
y_val_pred = clf.predict(X_val_std_scaled)


#metricas de las predicciones
print("todas")
print(classification_report(y_val, y_val_pred))

El resultado es mejor que cualquiera de los intentos individuales

Ahora se hará con los dos mejores resultados anteriores a la vez por si al agregar todos los dados se está perdiendo algo 

In [41]:
df = df.dropna(subset=["region", i])  # Drop rows where target or feature is NaN
X = df[["af_loudness","af_tempo"]].copy()
y = df["region"]

#primero separamos los datos de entrenamiento y validacion/test
X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(X,y,test_size=0.3,random_state=0,stratify=y)

# Luego separamos los datos de validación y pruebas                                       0.5 x 0.3 = 0.15
X_val, X_test, y_val, y_test = train_test_split(X_val_and_test, y_val_and_test, test_size=0.5, random_state=0, stratify=y_val_and_test)

#estandarizacion de datos, transformar los datos de manera que tengan media 0 y desviacion estandar 1.
std_scaler = StandardScaler()
X_train_std_scaled = std_scaler.fit_transform(X_train)

X_val_std_scaled = std_scaler.transform(X_val)
X_test_std_scaled = std_scaler.transform(X_test)

#definimos nuestro modelo
clf = DecisionTreeClassifier(criterion="gini",random_state=0)

#entrenamos al modelo con fit
clf.fit(X_train_std_scaled, y_train)   ## Entrenar usando X (features), y (clase)

#realizamos predicciones de nuestros datos con los datos de validación
y_val_pred = clf.predict(X_val_std_scaled)


#metricas de las predicciones
print("af_loudness y af_tempo")
print(classification_report(y_val, y_val_pred))

af_loudness y af_tempo
                precision    recall  f1-score   support

        Africa       0.94      0.48      0.64      1785
America_latina       0.82      0.75      0.78     15151
 America_norte       0.64      0.19      0.29      1785
          Asia       0.87      0.68      0.76      8921
        Europa       0.68      0.92      0.78     25448
       Oceania       0.83      0.25      0.39      1785
 Oriente_medio       0.86      0.38      0.52      4462

      accuracy                           0.75     59337
     macro avg       0.81      0.52      0.59     59337
  weighted avg       0.77      0.75      0.73     59337



La con mejor resultado sigue siendo loudness y key, al parecer al agregar el resto de los datos no suma ni resta al resultado por lo tanto tomares el de los dos mejores para el test.

In [44]:
#realizamos predicciones de nuestros datos con los datos de validación
y_test_pred = clf.predict(X_test_std_scaled)


#metricas de las predicciones
print("todas")
print(classification_report(y_test, y_test_pred))


todas
                precision    recall  f1-score   support

        Africa       0.94      0.46      0.62      1785
America_latina       0.82      0.75      0.78     15151
 America_norte       0.64      0.17      0.27      1785
          Asia       0.88      0.67      0.76      8922
        Europa       0.68      0.92      0.78     25448
       Oceania       0.83      0.22      0.35      1785
 Oriente_medio       0.87      0.38      0.53      4461

      accuracy                           0.74     59337
     macro avg       0.81      0.51      0.59     59337
  weighted avg       0.77      0.74      0.73     59337



Se podría seguir experimentando al buscar alguna otra combinación de características que den un mejor resultado, pero tomar demasiado tiempo al ser demasiado combinaciones